In [40]:
import pandas as pd

import os
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
import time
import joblib

MODEL_NAME = "NeuralNetwork"
ATTACK_NAME = "fault"
DATASET_NAME = "mapped_dataset.csv"

plots_dir = f'{ATTACK_NAME}/{MODEL_NAME}/{DATASET_NAME}/plots'
results_dir = f'{ATTACK_NAME}/{MODEL_NAME}/{DATASET_NAME}/results'
models_dir = f'{ATTACK_NAME}/{MODEL_NAME}/{DATASET_NAME}/models'

df = pd.read_csv('../../dataset/fault/mapped_dataset.csv')
df[[f'{i}' for i in range(10)]] = 1*pd.get_dummies(df['fault'])
df.pop('fault')
df

,max,min,mean,sd,rms,skewness,kurtosis,crest,form,0,1,2,3,4,5,6,7,8,9
0,0.35986,-0.41890,0.017840,0.122746,0.124006,-0.118571,-0.042219,2.901946,6.950855,1,0,0,0,0,0,0,0,0,0
1,0.46772,-0.36111,0.022255,0.132488,0.134312,0.174699,-0.081548,3.482334,6.035202,1,0,0,0,0,0,0,0,0,0
2,0.46855,-0.43809,0.020470,0.149651,0.151008,0.040339,-0.274069,3.102819,7.376926,1,0,0,0,0,0,0,0,0,0
3,0.58475,-0.54303,0.020960,0.157067,0.158422,-0.023266,0.134692,3.691097,7.558387,1,0,0,0,0,0,0,0,0,0
4,0.44685,-0.57891,0.022167,0.138189,0.139922,-0.081534,0.402783,3.193561,6.312085,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2295,0.21425,-0.19839,0.010769,0.064100,0.064983,-0.212497,-0.119312,3.297037,6.034174,0,0,0,0,0,0,1,0,0,0
2296,0.21967,-0.20882,0.013136,0.068654,0.069883,-0.061308,-0.295122,3.143410,5.319958,0,0,0,0,0,0,1,0,0,0
2297,0.20799,-0.21613,0.012571,0.067128,0.068279,-0.154754,-0.071405,3.046161,5.431299,0,0,0,0,0,0,1,0,0,0
2298,0.21425,-0.22405,0.012608,0.066813,0.067977,-0.326966,0.023662,3.151821,5.391672,0,0,0,0,0,0,1,0,0,0


In [2]:
x= df.drop([f'{i}' for i in range(10)], axis=1)
y = df[[f'{i}' for i in range(10)]]

x_trainval, x_test, y_trainval, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_trainval, y_trainval, test_size=0.2, random_state=42)

print('x_train', x_train.shape)
print('x_val',x_val.shape)
print('x_test',x_test.shape)
print('y_train', y_train.shape)
print('y_val',y_val.shape)
print('y_test',y_test.shape)

x_train (1288, 9)
x_val (322, 9)
x_test (690, 9)
y_train (1288, 10)
y_val (322, 10)
y_test (690, 10)


In [20]:
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.optimizers import Adam

model = Sequential()
model.add(InputLayer(input_shape=(9,)))
model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy')
history = model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=150)

Epoch 1/150
41/41 [==============================] - 0s 3ms/step - loss: 2.1817 - val_loss: 1.6703
Epoch 2/150
41/41 [==============================] - 0s 1ms/step - loss: 1.4676 - val_loss: 1.2400
Epoch 3/150
41/41 [==============================] - 0s 1ms/step - loss: 1.0817 - val_loss: 0.9222
Epoch 4/150
41/41 [==============================] - 0s 858us/step - loss: 0.8556 - val_loss: 0.7608
Epoch 5/150
41/41 [==============================] - 0s 1ms/step - loss: 0.7140 - val_loss: 0.6353
Epoch 6/150
41/41 [==============================] - 0s 1ms/step - loss: 0.6238 - val_loss: 0.5837
Epoch 7/150
41/41 [==============================] - 0s 1ms/step - loss: 0.5559 - val_loss: 0.5144
Epoch 8/150
41/41 [==============================] - 0s 1ms/step - loss: 0.5191 - val_loss: 0.5353
Epoch 9/150
41/41 [==============================] - 0s 1ms/step - loss: 0.4740 - val_loss: 0.4513
Epoch 10/150
41/41 [==============================] - 0s 1ms/step - loss: 0.4374 - val_loss: 0.4176
Epoch 1

In [21]:
h = history.history
import plotly.express as px
px.line(pd.DataFrame(h),y=['loss','val_loss'])

In [43]:
y_pred = model.predict(x_test)
for i in range(len(y_pred)):
    y_pred[i] = np.where(y_pred[i] == np.max(y_pred[i]),1,0)
y_pred
accuracy = accuracy_score(y_test, y_pred)
accuracy

def convert_y(y):
    new_y = []
    for i in range(len(y)):
        new_y.append(np.argmax(y[i]))
    return np.array(new_y)

#conf_matrix = confusion_matrix(convert_y(y_test.values), convert_y(y_pred))
conf_matrix = confusion_matrix(convert_y(y_test.values), convert_y(y_pred))
conf_matrix

22/22 [==============================] - 0s 712us/step


array([[63,  0,  1,  0,  0,  0,  0,  0,  6,  0],
       [ 0, 74,  1,  0,  0,  0,  1,  0,  2,  0],
       [ 0,  1, 62,  0,  0,  2,  0,  0,  9,  0],
       [ 0,  0,  0, 57,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 66,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 61,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, 61,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 72,  0,  0],
       [ 2,  2,  2,  0,  0,  0,  0,  0, 70,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  0, 74]], dtype=int64)

In [44]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets
